In [1]:
import numpy as np
import Plot as plot
import math
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
import time
import datetime
import pickle
import os
# import pysal
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib as plt
# from discreteMarkovChain import markovChain
import logging
from scipy import signal
import learningAlgs as classImportLA
import dataManipulation as dataMan
from itertools import permutations
import importlib
from datetime import timedelta
from multiprocessing import Pool
import multiprocessing
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('always')

In [2]:
address = input("the address of the collected data files (not alligned files or CSV files): ")

the address of the collected data files (not alligned files or CSV files): /home/netlab/Desktop/thesis/data/1node1-3-5/


In [3]:
fileNameArr = os.listdir(address)

extraArr = []
for i in range(len(fileNameArr)):
    if ".txt" not in fileNameArr[i]:
        extraArr.append(fileNameArr[i])

for i in range(len(extraArr)):
    fileNameArr.remove(extraArr[i])
    
print(fileNameArr)    



timerInMinute = 30 #chunking files to 30 minutes
timeInterval = 5 #seconds

CU_FileChunks = None
fileNameArr = ["500f80271400-1.txt"] #for test we are giving a ready file

LA = classImportLA.learningAlgs() #calling an object of the class

#*************checking if we have the processed version of file in our CSV or not, if not we are gonna process data otherwise just we gonna read it
pathFile = address + "/CSV/" + str(fileNameArr[0]) + ".csv"
importlib.reload(dataMan) #reload the class if it has cache (.pyc) to load the class from scratch
if os.path.isfile(pathFile) == False:
        print("we do not have processed data for file " + str(fileNameArr[0]) + " so we are making it")
        dataMan.normalDataSplitting(fileNameArr[0], 0, 0, timeInterval, address)

# print(fileNameArr[0])
# print(address)
stat, data = LA.csvChecker(fileNameArr[0], 0, address)        


print("now we have the processed data from pandas")
data["CU/255"] = data["CU"] / 255 #we add a column to our pandas table

secondsPerChunk = int(60 * int(timerInMinute)) + 1

data["timeIndex"] = -1
startIndex = 0
timeIndexVal = 1

print("chunking times in pandas")

#*******chunking data based on the time and put a value for its column
data["timeIndex"] = data["time"].apply(lambda x: math.floor(((x - data["time"][0]).seconds + ((x - data["time"][0]).days * 3600 * 24)) / secondsPerChunk))

numberOfChunks = data["timeIndex"][len(data) - 1] + 1


['500f801cede2.txtCH1', '500f801cf025.txtCH1', '500f801cf9c6.txtCH1', '500f8022b883.txtCH1', '40017acf5182.txtCH1', '500f8022abe3.txtCH1', '40017ad6b6a0.txtCH1', '500f801e2802.txtCH1', '500f801e28a2.txtCH1', '500f8022b121.txtCH1', '500f8022ab40.txtCH1', '500f80271400.txtCH1', '40017ad6b2e0.txtCH1', '500f801cf225.txtCH1', '40017ad6b624.txtCH1', '500f80271400-1.txt', '500f8022aca1.txtCH1']
here
500f80271400-1.txt is in csvChecker

we have the csv file: pulling out data

now we have the processed data from pandas
chunking times in pandas


In [4]:
cuTrans = np.zeros(shape=(numberOfChunks, 51, 51))
start = -1
next = -1
prevChunkVal = -1
newChunkVal = -1
firstIndexOfChunk = -1


cuTrans = np.zeros(shape=(numberOfChunks, 51, 51))
cuDifTrans = np.zeros(shape=(numberOfChunks, 51, 51))

start = -1
next = -1
prevChunkVal = -1
newChunkVal = -1
firstIndexOfChunk = -1
indexesOfChunks = data.index[data["timeIndex"] == i]
counter = 0
for index, row in data.iterrows():
    # if index % 10000 == 0:
        # print(index)
    newChunkVal = row["timeIndex"]
    if newChunkVal == prevChunkVal:
        start = next
        next = row["CU/255"] / 0.02
        if start != -1:
            cuTrans[newChunkVal, math.floor(start), math.floor(next)] += 1
        prevChunkVal = newChunkVal

    elif newChunkVal != prevChunkVal:
        if firstIndexOfChunk != -1:
            start = next
            next = data["CU/255"][firstIndexOfChunk] / 0.02
            cuTrans[prevChunkVal, math.floor(start), math.floor(next)] += 1
        firstIndexOfChunk = index
        next = row["CU/255"] / 0.02
        prevChunkVal = newChunkVal
indexesOfChunks = data.index[data["timeIndex"] == i]

print("doing lambda operation")
#this is only for this dataSet, so it is for "test"
data["time"] = data["time"].apply(lambda x: x + timedelta(hours = -7)) #our data is in UTC so we should make it to Moutain time

print("reading indices")
#**********we wanna remove those 30 minutes with less than 280 CU values *******************
removableIndices = []
for i in range(cuTrans.shape[0]):
    if (np.sum(cuTrans[i]) < 280):
        removableIndices.append(i)
 

doing lambda operation
reading indices


In [7]:
data["time"][0].second + data["time"][0].minute * 60 + data["time"][0].hour * 3600

68492

In [5]:
data["timeIndex"] = data["time"].apply(lambda x: math.floor(((x.second + (x.minute * 60) + (x.hour * 3600)) / 1800)))

In [ ]:
data["time"].loc[data["timeIndex"] == 48]

In [7]:
data.head()

,col1,CU,time,CU/255,timeIndex
0,0,45,2018-11-12 19:01:32,0.176471,38
1,1,45,2018-11-12 19:01:38,0.176471,38
2,2,51,2018-11-12 19:01:44,0.200000,38
3,3,53,2018-11-12 19:01:50,0.207843,38
4,4,45,2018-11-12 19:01:56,0.176471,38


In [6]:
timeIndexArr = [0 for i in range(48)]
def indexingForTest(x):
    if x["col1"] == 0:
        return 0
    elif data["timeIndex"][x["col1"] - 1] != x["timeIndex"]:
        timeIndexArr[data["timeIndex"][x["col1"] - 1]] += 1
        return timeIndexArr[x["timeIndex"]]
    else:
        return timeIndexArr[x["timeIndex"]]
data["train"] = data.apply(lambda x: indexingForTest(x), axis = 1)

In [8]:
data.loc[data["timeIndex"] == 28]

,col1,CU,time,CU/255,timeIndex,train
10698,10698,79,2018-11-13 14:00:02,0.309804,28,0
10699,10699,81,2018-11-13 14:00:08,0.317647,28,0
10700,10700,76,2018-11-13 14:00:14,0.298039,28,0
10701,10701,99,2018-11-13 14:00:20,0.388235,28,0
10702,10702,63,2018-11-13 14:00:26,0.247059,28,0
10703,10703,86,2018-11-13 14:00:32,0.337255,28,0
10704,10704,96,2018-11-13 14:00:38,0.376471,28,0
10705,10705,79,2018-11-13 14:00:44,0.309804,28,0
10706,10706,76,2018-11-13 14:00:50,0.298039,28,0
10707,10707,114,2018-11-13 14:00:56,0.447059,28,0


In [136]:
a = set(dataCpy["train"].values)
print(a)

b = set(data["train"].values)
print(b)

{0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37}


In [7]:
dataCpy = data
numberOfChunks = 48
cuTrans = np.zeros(shape=(5, numberOfChunks, 51, 51))
start = -1
next = -1
prevChunkVal = -1
newChunkVal = -1
firstIndexOfChunk = -1



cuTrans = np.zeros(shape=(numberOfChunks, 26, 26))
cuDifTrans = np.zeros(shape=(numberOfChunks, 26, 26))

start = -1
next = -1
prevChunkVal = -1
newChunkVal = -1
firstIndexOfChunk = -1
indexesOfChunks = dataCpy.index[dataCpy["timeIndex"] == i]
counter = 0
for index, row in dataCpy.iterrows():
    # if index % 10000 == 0:
        # print(index)
    if row["train"] >= 36:
        continue
    newChunkVal = row["timeIndex"]
    if newChunkVal == prevChunkVal:
        start = next
        next = row["CU/255"] / 0.04
        if start != -1:
            cuTrans[newChunkVal, math.floor(start), math.floor(next)] += 1
        prevChunkVal = newChunkVal

    elif newChunkVal != prevChunkVal:
        if firstIndexOfChunk != -1:
            start = next
            next = dataCpy["CU/255"][firstIndexOfChunk] / 0.04
            cuTrans[prevChunkVal, math.floor(start), math.floor(next)] += 1
        firstIndexOfChunk = index
        next = row["CU/255"] / 0.04
        prevChunkVal = newChunkVal
indexesOfChunks = dataCpy.index[dataCpy["timeIndex"] == i]

In [13]:
import sys
np.set_printoptions(threshold=sys.maxsize)
cuTrans.shape

(48, 26, 26)

In [8]:
print("normalizing")
ans = np.zeros(shape=(cuTrans.shape[0], 26))
for x in range(cuTrans.shape[0]):
    for i in range(cuTrans.shape[1]):
        sum = 0
        for j in range(cuTrans.shape[2]):
            sum += cuTrans[x][i][j]
        if sum != 0:
            cuTrans[x][i] = cuTrans[x][i]/sum

normalizing


In [9]:
def sectoring(permutedArr, sectorNumber):
#     print("aaaa")
    numOfSectorsIndex = []
    minLumpedError = math.inf
    bestLumped = [0]
    bestSectoring = []
    numbOfChunks = len(sectorNumber) - 1
#     print("here we look")
#     print(sectorNumber)

    
    
    sector = []
    indices = []
    lumpedArr = [[0 for i in range(numbOfChunks)] for i in range(numbOfChunks)]
    indicesArr = [[0 for i in range(numbOfChunks)] for i in range(numbOfChunks)]
    lumpedArrVal = [[0 for i in range(numbOfChunks)] for i in range(numbOfChunks)]
    quasiLumpedArr = [[0 for i in range(numbOfChunks)] for i in range(numbOfChunks)]
    fullLumpedArrIndices = [[0 for i in range(numbOfChunks)] for i in range(numbOfChunks)]
#     print(lumpedArr)
#     [[0 for i in range(len(permutedArr))] for i in range(len(permutedArr))]

#     print("initialized lumped Array is: " + str(lumpedArr))
    for x in range(1, len(sectorNumber)):
        index = x
        lumpedArr[x-1][x-1] = permutedArr[sectorNumber[x - 1] : sectorNumber[x], sectorNumber[x - 1] : sectorNumber[x]]
#         print("just this")
        a = np.arange(sectorNumber[x - 1], sectorNumber[x])
        b = np.arange(sectorNumber[x - 1], sectorNumber[x])
        A,B = np.meshgrid(a,b)
        AB=np.array([A.flatten(),B.flatten()]).T
        indicesArr[x - 1][x - 1] = AB
        sector.append(permutedArr[sectorNumber[x - 1] : sectorNumber[x], sectorNumber[x - 1] : sectorNumber[x]])
        index -= 1
        while index - 1 >= 0:
            lumpedArr[index - 1][x - 1] = permutedArr[sectorNumber[index - 1] : sectorNumber[index], sectorNumber[x - 1] : sectorNumber[x]]
#             print("just this")
            a = np.arange(sectorNumber[index - 1], sectorNumber[index])
            b = np.arange(sectorNumber[x - 1], sectorNumber[x])
            A,B = np.meshgrid(a,b)
            AB=np.array([A.flatten(),B.flatten()]).T
            indicesArr[index - 1][x - 1] = AB
            indices.append(AB)
#             fullLumpedArr[sectorNumber[index - 1] : sectorNumber[index], sectorNumber[x - 1] : sectorNumber[x]] = permutedArr[sectorNumber[index - 1] : sectorNumber[index], sectorNumber[x - 1] : sectorNumber[x]]
            #                     print("lumped Array is: " + str(lumpedArr))
#             print(fullLumpedArr)
            lumpedArr[x - 1][index - 1] = permutedArr[sectorNumber[x - 1] : sectorNumber[x], sectorNumber[index - 1] : sectorNumber[index]]
#             print("just this")
            a = np.arange(sectorNumber[x - 1], sectorNumber[x])
            b = np.arange(sectorNumber[index - 1], sectorNumber[index])
            A,B = np.meshgrid(a,b)
            AB=np.array([A.flatten(),B.flatten()]).T
            indicesArr[x - 1][index - 1] = AB
            indices.append(AB)
#             fullLumpedArr[sectorNumber[x - 1] : sectorNumber[x], sectorNumber[index - 1] : sectorNumber[index]] = permutedArr[sectorNumber[x - 1] : sectorNumber[x], sectorNumber[index - 1] : sectorNumber[index]]
#             print(fullLumpedArr)
#                     print("lumped Array is: " + str(lumpedArr))
            sector.append(permutedArr[sectorNumber[index - 1] : sectorNumber[index], sectorNumber[x - 1] : sectorNumber[x]])
            sector.append(permutedArr[sectorNumber[x - 1] : sectorNumber[x], sectorNumber[index - 1] : sectorNumber[index]])
            index -= 1
            
#     print(indicesArr)
#     print(lumpedArr)

    numOfRows = 0
    numOfRows = len(permutedArr)
    
    errorArr = [[0 for i in range(len(lumpedArr[0]))] for j in range(numOfRows)]
    semiLumpedArr = [[0 for i in range(len(lumpedArr[0]))] for j in range(numOfRows)]
    
#     print("error dimensions")
#     print(numOfRows)
#     print(len(lumpedArr[0]))
#     print(errorArr)
            
#     time.sleep(5)
#     print("we are here")
    probIter  = 0
    prob = [0 for i in range(len(permutedArr))]
    hitTimes = [0 for i in range(len(permutedArr))]
#     print("lumped arr is: " + str(lumpedArr))
    
    for l in range(len(lumpedArr)): #gets the numpy arrays
        sum = 0
        for x in range(len(lumpedArr[l])): #gets each numpy array
            probIter = 0
            for q in range(len(hitTimes)): #check the correct place of full matrix
                    if hitTimes[q] != len(permutedArr):
#                                 print("hit is " + str(hitTimes))
                        probIter = q
                        break
            quasiError = [0 for i in range(len(lumpedArr[l][x]))]
            
            if len(lumpedArr[l][x]) == 1:
                rowVal = 0
                for rows in range(0, l):
                    rowVal += len(lumpedArr[rows][0])
                errorArr[rowVal][x] = 0
                semiLumpedArr[rowVal][x] = np.sum(lumpedArr[l][x][0])
            
            else:
                minVal = math.inf
                for p in range(len(lumpedArr[l][x])):
                    if np.sum(lumpedArr[l][x][p]) < minVal:
                        minVal = np.sum(lumpedArr[l][x][p])
                
                for p in range(len(lumpedArr[l][x])):
                    rowVal = 0
                    for rows in range(0, l):
                        rowVal += len(lumpedArr[rows][0])
                    rowVal += p
                    
                    errorArr[rowVal][x] = np.sum(lumpedArr[l][x][p]) - minVal
                    semiLumpedArr[rowVal][x] = np.sum(lumpedArr[l][x][p])
                    
                    
            for p in range(len(lumpedArr[l][x])): #read rows              
                
                quasiError[p] = np.sum(lumpedArr[l][x][p])


                prob[probIter] += np.sum(lumpedArr[l][x][p])
#                         print("prob is: " + str(prob))
                hitTimes[probIter] += len(lumpedArr[l][x][p])
                probIter += 1
#             time.sleep(10)

#                     print(quasiError)
            minOfMaxErrs = math.inf
            lowErrIndex = math.inf
            if len(quasiError) > 1:
#                 print("quasi Error is: " + str(quasiError))
                for first in range(len(quasiError)):
                    maxErr = 0
                    for sec in range(len(quasiError)):
                        if math.fabs(quasiError[first] - quasiError[sec]) > maxErr:
                            maxErr = math.fabs(quasiError[first] - quasiError[sec])
                    if maxErr < minOfMaxErrs:
                        lowErrIndex = first
                        minOfMaxErrs = maxErr
            else:
                lowErrIndex = 0
                minOfMaxErrs = 0
            lumpedArrVal[l][x] = np.sum(lumpedArr[l][x][lowErrIndex])
#             print(minOfMaxErrs)
            quasiLumpedArr[l][x] = minOfMaxErrs
    
#     print("arrays:")
#     print(errorArr)
#     print(semiLumpedArr)
    
#     print("error is: ")
    maxError = - math.inf
    for i in range(len(errorArr)):
        if np.sum(errorArr[i]) > maxError:
            maxError = np.sum(errorArr[i])
#         print(np.sum(semiLumpedArr[i]))
#     print(maxError)

    return maxError, lumpedArrVal, sectorNumber



In [10]:
def combinationEnum(npArray):
    arr = []
    arrayResults = []
    minErr = math.inf
    lumpedMat = []
    sector = []
    permutationSave = []
    permuteMatrix = []
    indexList = []
    numOfSectorsIndex = []
    error = math.inf
    lumpedMatrix = []
    sectoringModel = []
    permutationModel = []
    
    
#     for i in range(len(classifier)):
#         arr.append(i)
    
#     p = 0
#     for x in range(len(classifierArr)):
#         indexList.append([])
#         for j in range(len(classifierArr[x])):
#             indexList[x].append(p)
#             p += 1
 

    matrixPlace = []
    newMatrixPlace = []
#     listIndex = list(subset)
    colEx = npArray


#             time.sleep(5)
    numOfSectorsIndex = []
#             print("even here")
    for i in range(1, len(npArray)): #we dont care about the first and the last raw and column because they are already in
        numOfSectorsIndex.append(i)
#             print("and here")
#             print(numOfSectorsIndex)
    listOfSectors = [] #we use it to prevent repitative sectors to operate
#             print(len(classifier))
    for i in range(1, 5):
#                 print("i is: " + str(i) )
#                 print("num of sector indices are: " + str(numOfSectorsIndex))
        extraArr = []
        for subset in permutations(numOfSectorsIndex, i): #drawing line for every possible index untill 5 classes
#                     print("i is: " + str(i) )
            if sorted(list(subset)) not in extraArr:
#                     print("subset list is: " + str(list(subset)))
#                         print("arrays of subset is: " + str(extraArr))
                extraArr.append(sorted(list(subset)))
#                     print("extraArr is: " + str(extraArr))
                sectorNumber = [0]
                for j in range(len(sorted(list(subset)))):
#                         print(extraArr[j])
#                         print("extraArr j is: " + str(extraArr[j]))                        
#                         for l in range(len(extraArr[j])):
                    sectorNumber.append(sorted(list(subset))[j])
#                         sectorNumber.extend(extraArr[j])
                sectorNumber.append(len(colEx))
#                     print("sector number is: " + str(sectorNumber))
#                     print("list of sectors are: " + str(listOfSectors))
                if sectorNumber not in listOfSectors:
#                             print("injam")
                    listOfSectors.append(sectorNumber)
        
    func = partial(sectoring, colEx)
    p = multiprocessing.Pool(multiprocessing.cpu_count())
    result = p.map(func, listOfSectors)
    p.close()
    p.join()
    minError = np.inf
    bestLumped = []
    bestClustering = []
    for i in range(len(result)):
        if result[i][0] <= 0.2:
            arr.append(result[i])
#             print(result[i][0], result[i][1], result[i][2] )
#         if result[i][0] < minError and result[i][0] != 0:
#             minError = result[i][0]
#             bestLumped = result[i][1]
#             bestClustering = result[i][2]
#         print(minError)
#         print(bestLumped)
#         print(bestClustering)
#     return minError, bestLumped, bestClustering
    return arr

In [14]:
from itertools import permutations
import itertools
from functools import partial
cuTrans_cpy = cuTrans
rowArg = []
colArg = []
y = 0
z = 0
x = 28
errorsArr = []
lumpedArr = []
sectorsArr = []
remainedClasses = []
results = []

#*****for finding the classes that are remained untouched in the process of reducing the transition matrix *****
for i in range(len(cuTrans_cpy[0])):
    remainedClasses.append(i)

# savedStat = [0 for i in range(len(allHours))]

counter = 0
# print(len(allHours))
# for index in range(len(checkList)):
#     if counter > 1:
#         break
    
#     print(x)
#     x = allHours[8]
#     counter += 1
for x in range(48):
    print(x)
    rowArg = []
    colArg = []
    for i in range(len(cuTrans_cpy[x])):
        if np.sum(cuTrans_cpy[x][i]) == 0:
            rowArg.append(i)
        if np.sum((cuTrans_cpy[x].T)[i] == 0):
            colArg.append(i)
            
    deleteList = list(set(rowArg) & set(colArg))
    sizeOfNewArr = len(deleteList)
#     print(deleteList)
    newArr = np.zeros(shape=(26-sizeOfNewArr, 26-sizeOfNewArr))
    
#     print(x)
#     print(cuTrans_cpy[x].shape)
    z = 0
    y = 0
    for i in range(len(cuTrans_cpy[x])):
        if i not in deleteList: 
            for j in range(26):
                if j not in deleteList:
                    newArr[z][y] = cuTrans_cpy[x][i][j]
#                     print(i, j)
                    y += 1
            y = 0
            z += 1
#     print("the transition matrix is: " + str(newArr))
#     print("shape of matrix is: " + str(newArr.shape))
    
    
#     print(deleteList)
    uncommonList = list(set(remainedClasses) - set(deleteList))
    print(uncommonList)
#     listOfMerge = [6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42]
    
    classifierArr = []
    extraVar = math.inf
    prevIndex = 0
    
#     print(index, checkList[index], newArr.shape)
#     uncommonList = [0, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 44]
    result = combinationEnum(newArr)
    results.append(result)
#     print(result)
#     firstIndex = data["time"].loc[data["timeIndex"] == allHours[8]].index[0]
#     time = data["time"].loc[firstIndex]    
#     result = result + (uncommonList, time, )
#     savedStat[index] = result
#     print(result[0], result[2])

0
[0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18]
1
[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
2
[0, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14]
3
[0, 3, 4, 5, 6, 7, 9]
4
[0, 3, 4, 5, 6, 7, 9, 10, 11, 13, 14, 16, 21]
5
[0, 3, 4, 5, 6, 7, 8, 9, 13, 14, 19, 20]
6
[0, 3, 4, 5, 6, 7, 9, 10, 17, 22]
7
[0, 3, 4, 5, 6, 7, 8, 10]
8
[0, 3, 4, 5, 6, 7, 8, 12, 13, 14, 18, 20, 21]


Process ForkPoolWorker-35:
Process ForkPoolWorker-34:
Process ForkPoolWorker-33:
Process ForkPoolWorker-36:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/netlab/anacond

KeyboardInterrupt: 

  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/home/netlab/anaconda3/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt


In [235]:
results[21]

[(0.10000000000000019,
  [[0.41509433962264153, 0.5849056603773585],
   [0.037037037037037035, 0.9629629629629628]],
  [0, 1, 20]),
 (0.16666666666666677,
  [[0.9444444444444444, 0.05555555555555555], [0.8, 0.2]],
  [0, 16, 20]),
 (0.16666666666666666,
  [[0.9937888198757763, 0.006211180124223602],
   [0.8333333333333333, 0.16666666666666666]],
  [0, 18, 20]),
 (0.16666666666666696,
  [[0.9937888198757763, 0.006211180124223602],
   [0.6666666666666666, 0.3333333333333333]],
  [0, 19, 20]),
 (0.10000000000000017,
  [[0.41509433962264153, 0.03773584905660377, 0.5471698113207548],
   [0.037037037037037035, 0.0, 0.9629629629629629],
   [0.01639344262295082, 0.009523809523809525, 0.9672131147540985]],
  [0, 1, 2, 20]),
 (0.16666666666666682,
  [[0.41509433962264153, 0.1509433962264151, 0.4339622641509434],
   [0.037037037037037035, 0.3333333333333333, 0.6296296296296295],
   [0.009523809523809525, 0.1111111111111111, 0.9]],
  [0, 1, 3, 20]),
 (0.16666666666666677,
  [[0.41509433962264153, 0

In [11]:
dataCpy = data
arrExtra = [0]

def particling(x):
    if x["col1"] - 1 < 0:
        arrExtra[0] = x["time"]
        return 0
    
    elif x["timeIndex"] != dataCpy["timeIndex"][x["col1"] - 1]:
        arrExtra[0] = x["time"]
        return 0
    
    else:
        return ((x["time"] - arrExtra[0]).seconds)/6

dataCpy["periodParticle"] = dataCpy.apply(lambda x: particling(x), axis = 1)

dataCpy["weekDay"] = data["time"].apply(lambda x: x.weekday())

In [121]:
dataCpy = dataCpy.loc[dataCpy["weekDay"] <= 4]

In [123]:
dataCpy.loc[dataCpy["weekDay"] == 4]

,col1,CU,time,CU/255,timeIndex,train,weekDay,periodParticle
43451,43451,45,2018-11-16 00:00:02,0.176471,0,3,4,0.0
43452,43452,40,2018-11-16 00:00:08,0.156863,0,3,4,1.0
43453,43453,48,2018-11-16 00:00:14,0.188235,0,3,4,2.0
43454,43454,40,2018-11-16 00:00:20,0.156863,0,3,4,3.0
43455,43455,43,2018-11-16 00:00:26,0.168627,0,3,4,4.0
43456,43456,48,2018-11-16 00:00:32,0.188235,0,3,4,5.0
43457,43457,40,2018-11-16 00:00:38,0.156863,0,3,4,6.0
43458,43458,53,2018-11-16 00:00:44,0.207843,0,3,4,7.0
43459,43459,43,2018-11-16 00:00:50,0.168627,0,3,4,8.0
43460,43460,40,2018-11-16 00:00:56,0.156863,0,3,4,9.0


In [101]:
sampleIntervals = 6 #seconds
minuteSplit = 30 #minutes
numOfSamples = minuteSplit * 60 / sampleIntervals

# numberOfDays = len(numOfDays)


days = np.zeros(7)
numOfThirtyMinsPerDay = np.zeros(int((24 * 60) / minuteSplit)) #in this case 48
which6SecondsPerPeriod = np.zeros(int(minuteSplit * 60 / sampleIntervals)) #in this case 300

#numberOfArraysForLearning = np.zeros(numberOfDays * numOfThirtyMinsPerDay * which6SecondsPerMinute)
XArraysForLearning = []
YArraysForLearning = []
XArraysForTesting = []
YArraysForTesting = []

prevRowTrain = np.inf
prevCU = np.inf

reg = "l2"
solvers = "lbfgs"

# print(statArr[28][0])

def clustering(x, bounds):
    for i in range(len(bounds) - 1):
        if x > bounds[i] and x <= bounds[i+1]:
            return i
    return len(bounds)
 
    
# dataCpy["cuClass"] = data["CU"].apply(lambda x: clustering(x))
# dataCpy["cuClass"].head()



In [182]:
results[0] = a
vals = results.copy()


In [241]:
vals = results
extraArr = []
statArr = []
# print(results[21])
for i in range(len(vals)):
    jVal = 0
    error = np.inf
    for j in range(len(vals[i])):
        if vals[i][j][0] <= error:
            error = vals[i][j][0]
            transMatrix = vals[i][j][1]
            boundaryMatrix =vals[i][j][2]
            jVal = j
#             print(j)
#     print(i)
#     print((transMatrix, boundaryMatrix))
    statArr.append([])
    statArr[i].append((transMatrix, boundaryMatrix))
#     print(i)
#     print(statArr[i])
#     print("jval is")
#     print(jVal)
#     print(vals[i])
    
    extraArr.append(vals[i][jVal])
    
for i in range(len(vals)):
    error = np.inf
    for j in range(len(vals[i])):
        if (vals[i][j][0] <= error) and (vals[i][j] not in extraArr):
            error = vals[i][j][0]
            transMatrix = vals[i][j][1]
            boundaryMatrix =vals[i][j][2]
            jVal = j
    print(i)
    print("jval is")
    print(jVal)
#     print(vals[i])    
    print(statArr[i])
    statArr[i].append((transMatrix, boundaryMatrix))

for i in range(len(statArr)):
    print(i, statArr[i][1])


0
jval is
11
[([[0.9999999999999999, 0.0], [1.0, 0.0]], [0, 11, 12])]
1
jval is
7
[([[0.9999999999999999, 0.0], [1.0, 0.0]], [0, 11, 12])]
2
jval is
0
[([[0.9997360781208762, 0.0002639218791237794, 0.0, 0.0, 0.0], [0.5, 0.0, 0.5, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0, 0.0], [0.5, 0.0, 0.0, 0.0, 0.5]], [0, 7, 8, 9, 10, 11])]
3
jval is
0
[([[1.0, 0.0], [1.0, 0.0]], [0, 6, 7])]
4
jval is
3
[([[0.9999999999999999, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.5, 0.0, 0.0, 0.5]], [0, 8, 9, 10, 11])]
5
jval is
3
[([[0.9999999999999998, 0.0], [1.0, 0.0]], [0, 11, 12])]
6
jval is
16
[([[0.9999999999999999, 0.0], [1.0, 0.0]], [0, 9, 10])]
7
jval is
2
[([[0.375, 0.6250000000000001], [0.007874015748031496, 0.9921259842519685]], [0, 1, 6])]
8
jval is
2
[([[0.38095238095238093, 0.6190476190476191], [0.004582484725050916, 0.9954175152749491]], [0, 1, 13])]
9
jval is
10
[([[1.0, 0.0], [1.0, 0.0]], [0, 6, 7])]
10
jval is
5
[([[0.9999999999999999, 0.0], [1.0, 0.0]],

In [209]:
statArr[0]

[([[0.9999999999999999, 0.0], [1.0, 0.0]], [0, 11, 12]),
 ([[0.9997281131049482, 0.0002718868950516585, 0.0],
   [1.0, 0.0, 0.0],
   [1.0, 0.0, 0.0]],
  [0, 10, 11, 12])]

In [249]:
clf = LogisticRegression(penalty = reg, max_iter = 100000, random_state = 0, solver = solvers , multi_class = 'multinomial')

pd.options.mode.chained_assignment = None

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

print("training and testing LR")
# ****************LR training model********************
for i in range(48):
    XArraysForLearning = []
    YArraysForLearning = []
    XArraysForTesting = []
    YArraysForTesting = []
    iterPandas = dataCpy.loc[(dataCpy["train"] < 36) & (dataCpy["timeIndex"] == i)]
    for n in range(len(statArr[i])):
        bounds = [10 * j for j in statArr[i][n][1]]
        iterPandas["cuClass"] = iterPandas["CU"].apply(lambda x: clustering(x, bounds))
        for index, row in iterPandas.iterrows():
            if prevRowTrain != row["train"]:
                prevRowTrain = row["train"]
                prevCU = row["cuClass"]
                continue

            lastCU = np.array(prevCU)
            days[row["weekDay"]] = 1
            which6SecondsPerPeriod[int(row["periodParticle"])] = 1
            XArray = np.append(lastCU, days)
            XArray = np.append(XArray, which6SecondsPerPeriod)
            XArraysForLearning.append(XArray)
            YArraysForLearning.append(row["cuClass"])

            days[row["weekDay"]] = 0
            which6SecondsPerPeriod[int(row["periodParticle"])] = 0
            prevCU = row["cuClass"]

#********************LR testing********************        
    iterPandas = dataCpy.loc[(data["train"] >= 36) & (dataCpy["timeIndex"] == i)]
    for n in range(len(statArr[i])):
        bounds = [10 * j for j in statArr[i][n][1]]
        iterPandas["cuClass"] = iterPandas["CU"].apply(lambda x: clustering(x, bounds))
    #     print(iterPandas.head())
        for index, row in iterPandas.iterrows():
            if prevRowTrain != row["train"]:
                prevRowTrain = row["train"]
                prevCU = row["cuClass"]
                continue

            lastCU = np.array(prevCU)
            days[row["weekDay"]] = 1
            which6SecondsPerPeriod[int(row["periodParticle"])] = 1
            XArray = np.append(lastCU, days)
            XArray = np.append(XArray, which6SecondsPerPeriod)
            XArraysForTesting.append(XArray)
            YArraysForTesting.append(row["cuClass"])

            days[row["weekDay"]] = 0
            which6SecondsPerPeriod[int(row["periodParticle"])] = 0
            prevCU = row["cuClass"]
    #     print("fitting")
        clf.fit(XArraysForLearning, YArraysForLearning)
        Ypred = clf.predict(XArraysForTesting)
    #     print("accing")
        acc = accuracy_score(YArraysForTesting, Ypred)
        errorW = f1_score(YArraysForTesting, Ypred, average='weighted', labels=np.unique(Ypred))

        recallW = recall_score(YArraysForTesting, Ypred, average='weighted')

        precisionW = precision_score(YArraysForTesting, Ypred, average='weighted')   
        print(i, acc)
# #     print("f1_score" + str(errorW))
# #     print("recall_score" + str(recallW))
# #     print("precision_score" + str(precisionW))


    
print("testing lumping")    
#********************Lumping prediction model ********************    
for x in range(48):
    iterPandas = data.loc[(data["train"] >= 36) & (data["timeIndex"] == x)]
    
    for n in range(len(statArr[x])):
#         print(statArr[x])
#         print(statArr[x][n])
        bounds = [10 * i for i in statArr[x][n][1]]
        choiceArr = [i for i in range(len(statArr[x][n][1]) - 1)]
        prediction = np.inf
        trueCounter = 0
        wholeCounter = 0

        for j in range(len(statArr[x][n][0])):
    #         print(statArr[x][0][j])
            if np.sum(statArr[x][n][0][j]) != 1:
                extra = np.array(statArr[x][n][0][j])
                extra /= extra.sum()
                statArr[x][n][0][j] = extra.tolist()

        for index, row in iterPandas.iterrows():
            wholeCounter += 1
            for i in range(len(bounds) - 1):
                if row["CU"] > bounds[i] and row["CU"] <= bounds[i+1]:
                    if prediction == i:
                        trueCounter += 1
                    prediction = np.random.choice(choiceArr, 1, p=statArr[x][n][0][i])
                    break
    #     print(trueCounter)
    #     print(wholeCounter)
        print(x, trueCounter/wholeCounter)
#     break




#     break
        

            

training and testing LR
0 1.0
0 1.0
1 0.9637096774193549
1 0.9637826961770624
2 0.9929824561403509
2 0.9947460595446584
3 0.9074733096085409
3 0.9502664298401421
4 1.0
4 0.9982300884955753
5 0.9818181818181818
5 0.9836660617059891
6 0.9858156028368794
6 0.7592920353982301
7 0.9222614840989399
7 0.6843033509700176
8 1.0
8 0.9981916817359855
9 0.848
9 0.6407185628742516
10 0.942652329749104
10 0.9534883720930233
11 0.9424460431654677
11 0.8348294434470377
12 1.0
12 0.9982300884955753
13 0.9822064056939501
13 0.9804618117229129
14 0.9964664310954063
14 0.9964726631393298
15 1.0
15 0.9981785063752276
16 0.9859649122807017
16 0.9877408056042032
17 0.9259259259259259
17 0.933456561922366
18 1.0
18 1.0
19 1.0
19 1.0
20 1.0
20 1.0
21 1.0
21 0.9981981981981982
22 0.9965397923875432
22 0.9965457685664939
23 1.0
23 0.9982238010657194
24 0.9929078014184397
24 0.9929203539823008
25 0.9428571428571428
25 0.9536541889483066
26 0.9929078014184397
26 0.9929203539823008
27 0.9181494661921709
27 0.932504

In [50]:
statArr[0][0]
statArr[0][1]

[0, 1, 2, 15]

In [29]:
data.loc[(data["train"] > 30) & (data["timeIndex"] == 1)]

,col1,CU,time,CU/255,timeIndex,train
418973,418973,58,2018-12-14 00:30:02,0.227451,1,31
418974,418974,45,2018-12-14 00:30:08,0.176471,1,31
418975,418975,61,2018-12-14 00:30:14,0.239216,1,31
418976,418976,48,2018-12-14 00:30:20,0.188235,1,31
418977,418977,48,2018-12-14 00:30:26,0.188235,1,31
418978,418978,45,2018-12-14 00:30:32,0.176471,1,31
418979,418979,58,2018-12-14 00:30:38,0.227451,1,31
418980,418980,45,2018-12-14 00:30:44,0.176471,1,31
418981,418981,45,2018-12-14 00:30:50,0.176471,1,31
418982,418982,48,2018-12-14 00:30:56,0.188235,1,31


In [58]:
lumpedTransition = np.array([[[], [0,30], [30,35], [35, 185]], [[0,30], 0.4266666666666667, 0.0, 0.5733333333333334], [[30,35], 0.0, 0.0, 1.0], [[35,185],0.005378271782000717, 0.0007171029042667623, 0.9939046253137325]])
print(lumpedTransition)
prediction = [10]
trueCounter = 0
wholeCounter = 0
# 0.027027027027027046 [[0.18181818181818182, 0.0, 0.8181818181818182, 0.0, 0.0], 
#                       [0.0, 0.1, 0.9, 0.0, 0.0], 
#                       [0.002277904328018223, 0.0026749888542131075, 0.987012987012987, 0.011111111111111112, 0.020833333333333332], 
#                       [0.0, 0.0, 0.8571428571428571, 0.14285714285714285, 0.0], 
#                       [0.0, 0.0, 0.6666666666666666, 0.0, 0.3333333333333333]] 
# [0, 1, 2, 19, 20, 21]
for index, row in data.iterrows():
    if row["timeIndex"] == 28 and row["train"] > 30:
        wholeCounter += 1
        value = row["CU"]
#         print(value)
        if value < 10:
            if prediction[0] == 0:
                trueCounter += 1
            prediction = np.random.choice([0,1,2,3,4],1, p=[0.18181818181818182, 0.8181818181818182, 0.0, 0.0, 0.0])
           
            
        elif value >= 10 and value < 20:
            if prediction[0] == 1:
                trueCounter += 1
            prediction = np.random.choice([0,1,2,3,4],1, p=[0.006407020146395633, 0.9381034317405196, 0.02488475530452884, 0.010645145324715116, 0.01995964748384084])
            
        elif value >= 20 and value < 190:
            if prediction[0] == 2:
                trueCounter += 1
            prediction = np.random.choice([0,1,2,3,4],1, p=[0.0, 0.9090909090909091, 0.09090909090909091, 0.0, 0.0])
        
        elif value >= 190 and value < 200:
            if prediction[0] == 3:
                trueCounter += 1
            prediction = np.random.choice([0,1,2,3,4],1, p=[0.0, 0.5714285714285714, 0.2857142857142857, 0.14285714285714285, 0.0])
        
        elif value >= 200 and value < 210:
            if prediction[0] == 4:
                trueCounter += 1
            prediction = np.random.choice([0,1,2,3,4],1, p=[0.0, 0.6666666666666666, 0.0, 0.0, 0.3333333333333333])
                    
    
print(wholeCounter)
print(trueCounter)
print(trueCounter/wholeCounter)
        

[[list([]) list([0, 30]) list([30, 35]) list([35, 185])]
 [list([0, 30]) 0.4266666666666667 0.0 0.5733333333333334]
 [list([30, 35]) 0.0 0.0 1.0]
 [list([35, 185]) 0.005378271782000717 0.0007171029042667623
  0.9939046253137325]]
1688
149
0.08827014218009478


In [132]:
print(result)

(0.017857142857142856, [[0.4266666666666667, 0.5733333333333334], [0.005378271782000717, 0.9946217282179992]], [0, 1, 24])


In [97]:
counter = 0
total = 0
for index, row in data.iterrows():
    if row["timeIndex"] == 3 and row["train"] > 30:
        total += 1
        if row["CU"] < 80:
            counter += 1
print(total)
print(counter)
print(counter / total)

1665
1665
1.0


In [129]:
lumpedTransition = np.array([[[], [0,30], [30,35], [35, 185]], [[0,30], 0.4266666666666667, 0.0, 0.5733333333333334], [[30,35], 0.0, 0.0, 1.0], [[35,185],0.005378271782000717, 0.0007171029042667623, 0.9939046253137325]])
print(lumpedTransition)
prediction = [10]
trueCounter = 0
wholeCounter = 0
for index, row in data.iterrows():
    if row["timeIndex"] == 0 and row["train"] > 30:
        wholeCounter += 1
        value = row["CU"]
#         print(value)
        if value < 30:
            if prediction[0] == 0:
                trueCounter += 1
            prediction = np.random.choice([0,1,2],1, p=[0.4266666666666667, 0.0, 0.5733333333333334])
           
            
        if value >= 30 and value < 35:
            if prediction[0] == 1:
                trueCounter += 1
            prediction = np.random.choice([0,1,2],1, p=[0.0, 0.0, 1.0])
            
        if value >= 35 and value < 185:
            if prediction[0] == 2:
                trueCounter += 1
            prediction = np.random.choice([0,1,2],1, p=[0.005378271782000717, 0.0007171029042667623, 0.9939046253137325])
    
print(wholeCounter)
print(trueCounter)
print(trueCounter/wholeCounter)
        

[[list([]) list([0, 30]) list([30, 35]) list([35, 185])]
 [list([0, 30]) 0.4266666666666667 0.0 0.5733333333333334]
 [list([30, 35]) 0.0 0.0 1.0]
 [list([35, 185]) 0.005378271782000717 0.0007171029042667623
  0.9939046253137325]]
1651
1623
0.9830405814657783


In [166]:
#calculating number of days
numOfDays = []
a = (data["time"][1].day, data["time"][1].year, data["time"][1].month)
def dateCalc(x):
    date = (x.year, x.month, x.day)
    if date not in numOfDays:
        numOfDays.append(date)
data["time"].apply(lambda x: dateCalc(x))

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
496738    None
496739    None
496740    None
496741    None
496742    None
496743    None
496744    None
496745    None
496746    None
496747    None
496748    None
496749    None
496750    None
496751    None
496752    None
496753    None
496754    None
496755    None
496756    None
496757    None
496758    None
496759    None
496760    None
496761    None
496762    None
496763    None
496764    None
496765    None
496766    None
496767    None
Name: time, Length: 496768, dtype: object

In [198]:
arrExtra = [0]

def particling(x):
    if x["col1"] - 1 < 0:
        arrExtra[0] = x["time"]
        return 0
    
    elif x["timeIndex"] != dataCpy["timeIndex"][x["col1"] - 1]:
        arrExtra[0] = x["time"]
        return 0
    
    else:
        return ((x["time"] - arrExtra[0]).seconds)/6

dataCpy["periodParticle"] = dataCpy.apply(lambda x: particling(x), axis = 1)

In [203]:
np.min(dataCpy["periodParticle"])

0.0

In [192]:
data.head(100)

,col1,CU,time,CU/255,timeIndex,train,cuClass,weekDay
0,0,45,2018-11-12 19:01:32,0.176471,38,0,2,0
1,1,45,2018-11-12 19:01:38,0.176471,38,0,2,0
2,2,51,2018-11-12 19:01:44,0.200000,38,0,2,0
3,3,53,2018-11-12 19:01:50,0.207843,38,0,2,0
4,4,45,2018-11-12 19:01:56,0.176471,38,0,2,0
5,5,45,2018-11-12 19:02:02,0.176471,38,0,2,0
6,6,45,2018-11-12 19:02:08,0.176471,38,0,2,0
7,7,48,2018-11-12 19:02:14,0.188235,38,0,2,0
8,8,58,2018-11-12 19:02:20,0.227451,38,0,2,0
9,9,53,2018-11-12 19:02:26,0.207843,38,0,2,0


In [12]:
dataCpy = data
def clustering(x):
    if x < 10:
        return 0
    elif x >= 10 and x < 20:
        return 1
    elif x >= 20 and x < 190:
        return 2
    elif x >= 190 and x < 200:
        return 3
    elif x >= 200 and x < 210:
        return 4
    else:
        return 5
 
    
dataCpy["cuClass"] = data["CU"].apply(lambda x: clustering(x))
dataCpy["cuClass"].head()

0    2
1    2
2    2
3    2
4    2
Name: cuClass, dtype: int64

In [13]:
dataCpy["weekDay"] = data["time"].apply(lambda x: x.weekday())

In [190]:
np.min(dataCpy["timeIndex"])

0

In [225]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans

sampleIntervals = 6 #seconds
minuteSplit = 30 #minutes
numOfSamples = minuteSplit * 60 / sampleIntervals

numberOfDays = len(numOfDays)


days = np.zeros(7)
numOfThirtyMinsPerDay = np.zeros(int((24 * 60) / minuteSplit)) #in this case 48
which6SecondsPerPeriod = np.zeros(int(minuteSplit * 60 / sampleIntervals)) #in this case 300

#numberOfArraysForLearning = np.zeros(numberOfDays * numOfThirtyMinsPerDay * which6SecondsPerMinute)
XArraysForLearning = []
YArraysForLearning = []
XArraysForTesting = []
YArraysForTesting = []

prevRowTrain = np.inf
prevCU = np.inf

reg = "l2"
solvers = "lbfgs"
clf = LogisticRegression(penalty = reg, max_iter = 100000, random_state = 0, solver = solvers , multi_class = 'multinomial')

for i in range(48):
    i = 28
    for index, row in dataCpy.iterrows():
        if row["timeIndex"] == i and row["train"] <= 30:
            if prevRowTrain != row["train"]:
                prevRowTrain = row["train"]
                prevCU = row["cuClass"]
                continue

            lastCU = np.array(prevCU)
            days[row["weekDay"]] = 1
            which6SecondsPerPeriod[int(row["periodParticle"])] = 1
            XArray = np.append(lastCU, days)
            XArray = np.append(XArray, which6SecondsPerPeriod)
            XArraysForLearning.append(XArray)
            YArraysForLearning.append(row["cuClass"])

            days[row["weekDay"]] = 0
            which6SecondsPerPeriod[int(row["periodParticle"])] = 0
            prevCU = row["cuClass"]


        if row["timeIndex"] == i and row["train"] > 30:
            if prevRowTrain != row["train"]:
                prevRowTrain = row["train"]
                prevCU = row["cuClass"]
                continue

            lastCU = np.array(prevCU)
            days[row["weekDay"]] = 1
            which6SecondsPerPeriod[int(row["periodParticle"])] = 1
            XArray = np.append(lastCU, days)
            XArray = np.append(XArray, which6SecondsPerPeriod)
            XArraysForTesting.append(XArray)
            YArraysForTesting.append(row["cuClass"])
        
            days[row["weekDay"]] = 0
            which6SecondsPerPeriod[int(row["periodParticle"])] = 0
            prevCU = row["cuClass"]
    
    clf.fit(XArraysForLearning, YArraysForLearning)
    Ypred = clf.predict(XArraysForTesting)
    acc = accuracy_score(YArraysForTesting, Ypred)
    print(acc)
    break
        

0.9898929845422116


In [226]:
YArraysForLearning

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 5,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [215]:


reg = "l2"
solvers = "lbfgs"
clf = LogisticRegression(penalty = reg, max_iter = 100000, random_state = 0, solver = solvers , multi_class = 'multinomial')
clf.fit(XArraysForLearning, YArraysForLearning)
Ypred = clf.predict(XArraysForTesting)
acc = accuracy_score(YArraysForTesting, Ypred)
print(acc)

In [216]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans

reg = "l2"
solvers = "lbfgs"
clf = LogisticRegression(penalty = reg, max_iter = 100000, random_state = 0, solver = solvers , multi_class = 'multinomial')
clf.fit(XArraysForLearning, YArraysForLearning)
Ypred = clf.predict(XArraysForTesting)

In [222]:
acc = accuracy_score(YArraysForTesting, Ypred)
print(acc)

0.9898929845422116


In [12]:
numberOfChunks = 48
# cuTrans = np.zeros(shape=(5, numberOfChunks, 51, 51))
start = -1
next = -1

fullTrans = np.zeros(shape=(26, 26))
# cuDifTrans = np.zeros(shape=(numberOfChunks, 26, 26))

start = -1
next = -1
prevChunkVal = -1
newChunkVal = -1
firstIndexOfChunk = -1
counter = 0
for index, row in dataCpy.iterrows():
    start = next
    next = row["CU/255"] / 0.04
    if start != -1:
        fullTrans[math.floor(start), math.floor(next)] += 1


In [13]:
print("normalizing")
normalizedFullTrans = np.zeros(shape=(26, 26))
for x in range(fullTrans.shape[0]):
    sum = 0
    if np.sum(fullTrans[x]) != 0:
        fullTrans[x] /= np.sum(fullTrans[x])
    

normalizing


In [16]:
fullTrans

array([[3.66155502e-01, 0.00000000e+00, 0.00000000e+00, 5.64971751e-02,
        2.46435297e-01, 2.15496368e-01, 5.29997310e-02, 2.58272801e-02,
        1.18375034e-02, 6.72585418e-03, 6.18778585e-03, 2.42130751e-03,
        2.69034167e-03, 1.88323917e-03, 8.07102502e-04, 1.07613667e-03,
        5.38068335e-04, 1.34517084e-03, 0.00000000e+00, 2.69034167e-04,
        5.38068335e-04, 2.69034167e-04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00

In [19]:
from itertools import permutations
import itertools
from functools import partial
# cuTrans_cpy = cuTrans
rowArg = []
colArg = []
y = 0
z = 0
x = 28
errorsArr = []
lumpedArr = []
sectorsArr = []
remainedClasses = []
results = []
fullMatResults = []

#*****for finding the classes that are remained untouched in the process of reducing the transition matrix *****
for i in range(len(fullTrans)):
    remainedClasses.append(i)

counter = 0

rowArg = []
colArg = []
for i in range(len(fullTrans)):
    if np.sum(fullTrans[i]) == 0:
        rowArg.append(i)
    if np.sum((fullTrans.T)[i] == 0):
        colArg.append(i)

deleteList = list(set(rowArg) & set(colArg))
sizeOfNewArr = len(deleteList)
#     print(deleteList)
newArr = np.zeros(shape=(26-sizeOfNewArr, 26-sizeOfNewArr))

#     print(x)
#     print(cuTrans_cpy[x].shape)
z = 0
y = 0
for i in range(len(fullTrans)):
    if i not in deleteList: 
        for j in range(26):
            if j not in deleteList:
                newArr[z][y] = fullTrans[i][j]
#                     print(i, j)
                y += 1
        y = 0
        z += 1
#     print("the transition matrix is: " + str(newArr))
#     print("shape of matrix is: " + str(newArr.shape))


#     print(deleteList)
uncommonList = list(set(remainedClasses) - set(deleteList))
print(uncommonList)
#     listOfMerge = [6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42]

classifierArr = []
extraVar = math.inf
prevIndex = 0

#     print(index, checkList[index], newArr.shape)
#     uncommonList = [0, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 44]
fullMatResult = combinationEnum(newArr)
fullMatResults.append(fullMatResult)
#     print(result)
#     firstIndex = data["time"].loc[data["timeIndex"] == allHours[8]].index[0]
#     time = data["time"].loc[firstIndex]    
#     result = result + (uncommonList, time, )
#     savedStat[index] = result
#     print(result[0], result[2])

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


In [20]:
minErr = np.inf
iIndex = np.inf
for i in range(len(fullMatResults[0])):
    print(fullMatResults[0][i][0], fullMatResults[0][i][2], i)


0.019607843137255276 [0, 1, 22] 0
0.012121212121212366 [0, 21, 22] 1
0.019607843137255165 [0, 1, 2, 22] 2
0.027450980392157154 [0, 1, 21, 22] 3
0.06930033987697862 [0, 1, 2, 3, 22] 4
0.027450980392157043 [0, 1, 2, 21, 22] 5
0.06422018348623884 [0, 19, 20, 21, 22] 6
0.06931314126977016 [0, 1, 2, 3, 21, 22] 7
0.07798165137614683 [0, 1, 19, 20, 21, 22] 8
0.17612346799818435 [0, 11, 19, 20, 21, 22] 9
0.1732418524871356 [0, 12, 19, 20, 21, 22] 10
0.1963589864721519 [0, 13, 19, 20, 21, 22] 11
0.15880704579125296 [0, 14, 19, 20, 21, 22] 12
0.15318242522669417 [0, 15, 19, 20, 21, 22] 13
0.11111111111111135 [0, 16, 19, 20, 21, 22] 14
0.09358752166377826 [0, 17, 19, 20, 21, 22] 15
0.09137055837563485 [0, 18, 19, 20, 21, 22] 16


In [14]:
dataFull = data
def clustering(x):
    if x < 100:
        return 0
    elif x >= 100 and x < 190:
        return 1
    elif x >= 190 and x < 200:
        return 2
    elif x >= 200 and x < 210:
        return 3
    elif x >= 210 and x < 220:
        return 4
    else:
        return 5
 
    
dataFull["cuClass"] = dataFull["CU"].apply(lambda x: clustering(x))
dataFull["weekDay"] = dataFull["time"].apply(lambda x: x.weekday())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans

sampleIntervals = 6 #seconds
minuteSplit = 30 #minutes
numOfSamples = minuteSplit * 60 / sampleIntervals

# numberOfDays = len(numOfDays)


days = np.zeros(7)
numOfThirtyMinsPerDay = np.zeros(int((24 * 60) / minuteSplit)) #in this case 48
which6SecondsPerPeriod = np.zeros(int(minuteSplit * 60 / sampleIntervals)) #in this case 300

#numberOfArraysForLearning = np.zeros(numberOfDays * numOfThirtyMinsPerDay * which6SecondsPerMinute)
XArraysForLearning = []
YArraysForLearning = []
XArraysForTesting = []
YArraysForTesting = []

prevRowTrain = np.inf
prevCU = 0

reg = "l2"
arrL = [1, 5, 10]
solvers = "lbfgs"
clf = LogisticRegression(penalty = reg, max_iter = 100000, random_state = 0, solver = solvers , multi_class = 'multinomial')

# for i in range(48):
#     i = 28
for HLNumber in arrL:
    print("aaaaaaaa")
    if HLNumber == 1:
        clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100,), random_state=1)

    if HLNumber == 5:
        clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 100, 100, 100, 100), random_state=1)

    if HLNumber == 10:
        clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100, 100, 100), random_state=1)

    iterPandas = dataFull.loc[dataFull["train"] >= 10]
    for index, row in iterPandas.iterrows():

        lastCU = np.array(prevCU)
        days[row["weekDay"]] = 1
        which6SecondsPerPeriod[int(row["periodParticle"])] = 1
        numOfThirtyMinsPerDay[int(row["timeIndex"])] = 1
        XArray = np.append(lastCU, days)
        XArray = np.append(XArray, which6SecondsPerPeriod)
        XArray = np.append(XArray, numOfThirtyMinsPerDay)
        XArraysForLearning.append(XArray)
        YArraysForLearning.append(row["cuClass"])

        days[row["weekDay"]] = 0
        which6SecondsPerPeriod[int(row["periodParticle"])] = 0
        numOfThirtyMinsPerDay[int(row["timeIndex"])] = 0
        prevCU = row["cuClass"]

    print("testing")
    iterPandas = dataFull.loc[dataFull["train"] < 10]
    for index, row in iterPandas.iterrows():
        lastCU = np.array(prevCU)
        days[row["weekDay"]] = 1
        which6SecondsPerPeriod[int(row["periodParticle"])] = 1
        numOfThirtyMinsPerDay[int(row["timeIndex"])] = 1
        XArray = np.append(lastCU, days)
        XArray = np.append(XArray, which6SecondsPerPeriod)
        XArray = np.append(XArray, numOfThirtyMinsPerDay)
        XArraysForTesting.append(XArray)
        YArraysForTesting.append(row["cuClass"])

        days[row["weekDay"]] = 0
        which6SecondsPerPeriod[int(row["periodParticle"])] = 0
        numOfThirtyMinsPerDay[int(row["timeIndex"])] = 0
        prevCU = row["cuClass"]

    print("fitting")
    print(len(XArraysForLearning))
    print(len(YArraysForLearning))
    clf.fit(XArraysForLearning, YArraysForLearning)
    Ypred = clf.predict(XArraysForTesting) 
    errorW = f1_score(YArraysForTesting, Ypred, average='micro', labels=np.unique(Ypred))

    recallW = recall_score(YArraysForTesting, Ypred, average='micro')

    precisionW = precision_score(YArraysForTesting, Ypred, average='micro')   
    acc = accuracy_score(YArraysForTesting, Ypred)
    print(HLNumber)
    print("f1score " + str(errorW))
    print("recall_score " + str(recallW))
    print("precision_score " + str(precisionW))
    print("acc" + str(acc))


aaaaaaaa
testing
fitting
361477
361477
1
f1score 0.9422430169043026
recall_score 0.9422430169043026
precision_score 0.9422430169043026
acc0.9422430169043026
aaaaaaaa
testing
fitting
722954
722954


In [15]:
dataFull

,col1,CU,time,CU/255,timeIndex,train,periodParticle,weekDay,cuClass
0,0,45,2018-11-12 19:01:32,0.176471,38,0,0.0,0,0
1,1,45,2018-11-12 19:01:38,0.176471,38,0,1.0,0,0
2,2,51,2018-11-12 19:01:44,0.200000,38,0,2.0,0,0
3,3,53,2018-11-12 19:01:50,0.207843,38,0,3.0,0,0
4,4,45,2018-11-12 19:01:56,0.176471,38,0,4.0,0,0
5,5,45,2018-11-12 19:02:02,0.176471,38,0,5.0,0,0
6,6,45,2018-11-12 19:02:08,0.176471,38,0,6.0,0,0
7,7,48,2018-11-12 19:02:14,0.188235,38,0,7.0,0,0
8,8,58,2018-11-12 19:02:20,0.227451,38,0,8.0,0,0
9,9,53,2018-11-12 19:02:26,0.207843,38,0,9.0,0,0


In [16]:
dataCpy

,col1,CU,time,CU/255,timeIndex,train,periodParticle,weekDay
0,0,45,2018-11-12 19:01:32,0.176471,38,0,0.0,0
1,1,45,2018-11-12 19:01:38,0.176471,38,0,1.0,0
2,2,51,2018-11-12 19:01:44,0.200000,38,0,2.0,0
3,3,53,2018-11-12 19:01:50,0.207843,38,0,3.0,0
4,4,45,2018-11-12 19:01:56,0.176471,38,0,4.0,0
5,5,45,2018-11-12 19:02:02,0.176471,38,0,5.0,0
6,6,45,2018-11-12 19:02:08,0.176471,38,0,6.0,0
7,7,48,2018-11-12 19:02:14,0.188235,38,0,7.0,0
8,8,58,2018-11-12 19:02:20,0.227451,38,0,8.0,0
9,9,53,2018-11-12 19:02:26,0.207843,38,0,9.0,0
